In [1]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(os.path.join(os.getcwd(), '..', '..', '..', 'pplSIT', 'workflow', 'utils'))

In [2]:
%matplotlib inline

import pandas as pd
from session.sessions import selected_009266, selected_008229, selected_009265
from imports import *
from scipy import stats
from scipy import signal
from sklearn.manifold import TSNE
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from statsmodels.formula.api import ols, glm
from regression import get_GLM_and_prediction

from Behavior.behavior import get_extent, get_idxs_behav_state
from population import unit_response_matrix, activity_at_phase

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [15]:
source = '/home/sobolev/nevermind_ag-grothe/AG_Pecka/data/processed/'
source = '/home/sobolev/nevermind/AG_Pecka/data/processed/'

sessions = [s for s in selected_009266.keys()]
#sessions = [s for s in selected_009265.keys()]
#sessions = [s for s in selected_008229.keys()]
sessions.sort()

selected = sessions[:]
try:
    selected.remove('009266_hippoSIT_2023-04-20_15-24-14')
except:
    pass
try:
    selected.remove('009265_hippoSIT_2023-02-27_10-18-32')
    selected.remove('009265_hippoSIT_2023-02-27_15-33-46')
except:
    pass
selected

['009266_hippoSIT_2023-04-17_17-04-17',
 '009266_hippoSIT_2023-04-18_10-10-37',
 '009266_hippoSIT_2023-04-18_17-03-10',
 '009266_hippoSIT_2023-04-19_10-33-51',
 '009266_hippoSIT_2023-04-20_08-57-39',
 '009266_hippoSIT_2023-04-21_08-43-00',
 '009266_hippoSIT_2023-04-21_13-12-31',
 '009266_hippoSIT_2023-04-24_10-08-11',
 '009266_hippoSIT_2023-04-24_16-56-55',
 '009266_hippoSIT_2023-04-26_08-20-17',
 '009266_hippoSIT_2023-05-02_12-22-14',
 '009266_hippoSIT_2023-05-04_19-47-15',
 '009266_hippoSIT_2023-05-22_09-27-22',
 '009266_hippoSIT_2023-05-23_09-18-05',
 '009266_hippoSIT_2023-05-25_15-55-57',
 '009266_hippoSIT_2023-06-14_08-21-23',
 '009266_hippoSIT_2023-06-19_08-58-35']

In [39]:
#session = selected[0]

conf_mxs = {}
scores   = {}
for session in selected[:]:
    ft = 'tSNE'
    fp = 70
    animal = session.split('_')[0]
    session_path     = os.path.join(source, animal, session)
    meta_file        = os.path.join(source, animal, session, 'meta.h5')
    moseq_file       = os.path.join(source, animal, session, 'MoSeq.h5')
    moseq_class_file = os.path.join(source, animal, session, 'analysis', 'MoSeq_tSNE_UMAP.h5')

    with h5py.File(meta_file, 'r') as f:
        tl = np.array(f['processed']['timeline'])
        tgt_mx = np.array(f['processed']['target_matrix'])
        events = np.array(f['processed']['sound_events'])
    with h5py.File(moseq_class_file, 'r') as f:
        idxs_srm_tl = np.array(f['idxs_srm_tl'])
        fit = np.array(f[ft][str(fp)])
    with h5py.File(moseq_file, 'r') as f:
        moseq = np.array(f['moseq'])

    # events filter
    idxs_filt_ev = np.where(events[:, 1] == 0)[0]  # silence filter
    idxs_filt_ev = np.where( (events[:, 1] == 0) | (events[:, 1] == 1) )[0]  # non-target filter
    #idxs_filt_ev = np.arange(len(events))  # no filter

    # 1. Speed
    width = 70  # 100 points ~= 1 sec with at 100Hz
    kernel = signal.gaussian(width, std=(width) / 7.2)
    dx = np.sqrt(np.square(np.diff(moseq[:, 3])) + np.square(np.diff(moseq[:, 4])))
    dt = np.diff(moseq[:, 0])
    speed = np.concatenate([dx/dt, [dx[-1]/dt[-1]]])
    speed_smooth = np.convolve(speed, kernel, 'same') / kernel.sum()
    speed_ev = speed_smooth[events[:, 2].astype(np.int32)]
    speed_filt_ev = speed_ev[idxs_filt_ev]

    # 2. Population activity 
    bins, unit_mx = unit_response_matrix(session_path, [1, 2], times_to_event=[])

    # z-score
    unit_act_matrix = unit_mx.T
    for u, unit_data in enumerate(unit_act_matrix):
        unit_act_matrix[u] = stats.zscore(unit_data)
    unit_mx = unit_act_matrix.T

    # take first PC from PCA on multiple units
    pca = decomposition.PCA(n_components=3)
    pca.fit(unit_mx)
    X = pca.transform(unit_mx)
    pop_act = X[:, 0]  # PC1 score

    # filter SILENCE
    pop_act_filt = pop_act[idxs_filt_ev]

    # smooth
    k_width = 30
    kernel  = signal.gaussian(k_width, std=(k_width) / 7.2)
    pop_act_filt = np.convolve(pop_act_filt, kernel, 'same') / kernel.sum()

    # filter slow oscillations
    sos = signal.butter(10, 0.001, fs=4, analog=False, btype='highpass', output='sos')
    pop_act_filt = signal.sosfiltfilt(sos, pop_act_filt)

    # just to test - W4 only
    w4 = activity_at_phase(session_path, phase=4, do_pca=True)
    w4_filt_ev = w4[idxs_filt_ev]


    # 3. Behavioral state
    idxs_tl_tgt_succ = []
    for tgt_rec in tgt_mx[tgt_mx[:, 4] == 1]:
        idxs_tl_tgt_succ += list(range(tgt_rec[2], tgt_rec[3]))
    idxs_tl_tgt_succ = np.array(idxs_tl_tgt_succ)
    idxs_AL_tl = get_idxs_behav_state(source, session, idxs_tl_tgt_succ, fit_type='tSNE', fit_parm=70, sigma=0.3, margin=10, bin_count=100)
    idxs_AL_ev = np.array([k for k, x in enumerate(events) if x[2] in idxs_AL_tl], dtype=np.int32)

    bs = np.zeros(len(events))
    bs[idxs_AL_ev] = 1
    bs_filt_ev = bs[idxs_filt_ev]

    # try smth random to show that random staff doesn't affect the score
    rnd = np.random.rand(len(bs_filt_ev))

    
    # 4. Compute LR and prediction
    combs = [
        np.column_stack([speed_filt_ev, w4_filt_ev]),
        np.column_stack([speed_filt_ev, w4_filt_ev, rnd]),
        np.column_stack([speed_filt_ev, rnd]),
        np.column_stack([w4_filt_ev, rnd]),
        np.column_stack([rnd])
    ]
    labels = ['Speed + W4', 'Speed + W4 + RND', 'Speed only', 'W4 only', 'RND']

    # 5a. Logistic regression
    # try to compensate for bias
    bs_high = np.where(bs_filt_ev > 0)[0]
    bs_low  = np.where(bs_filt_ev == 0)[0]
    if len(bs_high) != len(bs_low):
        idx_rem = 0 if len(bs_high) < len(bs_low) else 1
        idxs_exclude = []
        while len(idxs_exclude) < int(np.abs(len(bs_high) - len(bs_low))):
            idx = np.random.randint(len(bs_filt_ev))
            if bs_filt_ev[idx] == idx_rem and not idx in idxs_exclude:
                idxs_exclude.append(idx)
    idxs_include = np.array([x for x in range(len(bs_filt_ev)) if x not in idxs_exclude])

    print("%s - Scores: (AL: %.2f)" % (session[-19:-6], (len(idxs_include)/2)/len(bs_filt_ev)))
    
    for j, lr_data in enumerate(combs):
        X_train, X_test, y_train, y_test = train_test_split(lr_data[idxs_include], bs_filt_ev[idxs_include], test_size=0.33)
        model = LogisticRegression(solver='liblinear', random_state=0).fit(X_train, y_train)
        print("%.2f: %s" % (model.score(X_test, y_test), labels[j]))
        #print(confusion_matrix(y_test, model.predict(X_test)))
        
        if j == 0:
            scores[session]   = model.score(X_test, y_test)
            conf_mxs[session] = confusion_matrix(y_test, model.predict(X_test))
       
    # 5b. Use SVMs
    
    # 5c. Use GLMs
    #for j, lr_data in enumerate(combs):
    #    corr, pval, params, pvalues = get_GLM_and_prediction(lr_data, bs_filt_ev)
    #    print("%.2f, %.3f: %s" % (corr, pval, labels[j]))


2023-04-17_17 - Scores: (AL: 0.08)
0.74: Speed + W4
0.73: Speed + W4 + RND
0.68: Speed only
0.69: W4 only
0.47: RND
2023-04-18_10 - Scores: (AL: 0.22)
0.77: Speed + W4
0.77: Speed + W4 + RND
0.73: Speed only
0.71: W4 only
0.49: RND
2023-04-18_17 - Scores: (AL: 0.24)
0.69: Speed + W4
0.69: Speed + W4 + RND
0.68: Speed only
0.59: W4 only
0.49: RND
2023-04-19_10 - Scores: (AL: 0.48)
0.65: Speed + W4
0.63: Speed + W4 + RND
0.62: Speed only
0.60: W4 only
0.49: RND
2023-04-20_08 - Scores: (AL: 0.34)
0.72: Speed + W4
0.71: Speed + W4 + RND
0.68: Speed only
0.69: W4 only
0.49: RND
2023-04-21_08 - Scores: (AL: 0.41)
0.65: Speed + W4
0.63: Speed + W4 + RND
0.63: Speed only
0.58: W4 only
0.51: RND
2023-04-21_13 - Scores: (AL: 0.47)
0.57: Speed + W4
0.56: Speed + W4 + RND
0.57: Speed only
0.56: W4 only
0.49: RND
2023-04-24_10 - Scores: (AL: 0.38)
0.68: Speed + W4
0.70: Speed + W4 + RND
0.66: Speed only
0.65: W4 only
0.49: RND
2023-04-24_16 - Scores: (AL: 0.34)
0.64: Speed + W4
0.63: Speed + W4 + R

In [40]:
for k, v in conf_mxs.items():
    print("%s: score %.2f, FP/FN: %.2f" % (session[-19:-6], scores[k], v[0][1]/v[1][0]))

2023-06-19_08: score 0.74, FP/FN: 1.84
2023-06-19_08: score 0.77, FP/FN: 1.62
2023-06-19_08: score 0.69, FP/FN: 2.34
2023-06-19_08: score 0.65, FP/FN: 1.48
2023-06-19_08: score 0.72, FP/FN: 1.71
2023-06-19_08: score 0.65, FP/FN: 2.10
2023-06-19_08: score 0.57, FP/FN: 2.49
2023-06-19_08: score 0.68, FP/FN: 1.21
2023-06-19_08: score 0.64, FP/FN: 3.42
2023-06-19_08: score 0.76, FP/FN: 1.28
2023-06-19_08: score 0.70, FP/FN: 1.64
2023-06-19_08: score 0.72, FP/FN: 1.47
2023-06-19_08: score 0.67, FP/FN: 2.15
2023-06-19_08: score 0.72, FP/FN: 1.79
2023-06-19_08: score 0.68, FP/FN: 1.38
2023-06-19_08: score 0.70, FP/FN: 4.27
2023-06-19_08: score 0.68, FP/FN: 1.52
